In [1]:
from google.colab import drive

import shutil

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install gradio
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, AutoModelWithLMHead


import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

from tqdm.auto import tqdm

from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

import re

In [4]:
tokenizer_t5_base = AutoTokenizer.from_pretrained("t5-base")
tokenizer_t5_small = AutoTokenizer.from_pretrained("t5-small")
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
tokenizer_gpt2_medium = AutoTokenizer.from_pretrained("gpt2-medium")

checkpoint_t5_base = torch.load('drive/MyDrive/best_t5_base_model.pt', map_location=torch.device('cpu'))
checkpoint_t5_small = torch.load('drive/MyDrive/best_t5_small_model.pt', map_location=torch.device('cpu'))
checkpoint_gpt2_model = torch.load('drive/MyDrive/best_gpt2_model.pt', map_location=torch.device('cpu'))
checkpoint_gpt2_medium_model = torch.load('drive/MyDrive/best_gpt2_medium_model.pt', map_location=torch.device('cpu'))

model_t5_base=AutoModelWithLMHead.from_pretrained("t5-base", pad_token_id=tokenizer_t5_base.eos_token_id)
model_t5_small=AutoModelWithLMHead.from_pretrained("t5-small", pad_token_id=tokenizer_t5_small.eos_token_id)
model_gpt2=AutoModelWithLMHead.from_pretrained("gpt2", pad_token_id=tokenizer_gpt2.eos_token_id)
model_gpt2_medium=AutoModelWithLMHead.from_pretrained("gpt2-medium", pad_token_id=tokenizer_gpt2_medium.eos_token_id)

model_t5_base.load_state_dict(checkpoint_t5_base)
model_t5_small.load_state_dict(checkpoint_t5_small)
model_gpt2.load_state_dict(checkpoint_gpt2_model)
model_gpt2_medium.load_state_dict(checkpoint_gpt2_medium_model)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behav

<All keys matched successfully>

In [5]:

import gradio as gr
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:


def greet(sentence, gpt2, gpt2_medium, t5_base, t5_small, temperature):
    data_sentence_list=tokenize.sent_tokenize(sentence)

    res=""
    if gpt2:
      model=model_gpt2
      tokenizer=tokenizer_gpt2
    if gpt2_medium:
      model=model_gpt2_medium
      tokenizer=tokenizer_gpt2_medium
    if t5_base:
      model=model_t5_base
      tokenizer=tokenizer_t5_base
    if t5_small:
      model=model_t5_small
      tokenizer=tokenizer_t5_small


    for sentence in data_sentence_list:
      input = tokenizer.encode(sentence)
      output = model.generate(input_ids=torch.tensor([input]).to("cpu"), max_new_tokens=40, do_sample=True, temperature=temperature)
      output = output[0]
      output = output[1:-1]
      out_str=tokenizer.decode(output)
      if gpt2 or gpt2_medium:
        out_str_list=out_str.split("=")
        out_str=out_str_list[1]
      res=res+" "+out_str

    return  res

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", "checkbox", "checkbox", "checkbox", gr.Slider(0, 5, 0.1)],
    outputs=["text"],
)

demo.launch(debug=True)   

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
